In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import warnings
from datasets import load_dataset
import os
import json
import joblib
from datetime import datetime

warnings.filterwarnings('ignore')

class CustomerSupportChatbot:
    def __init__(self):
        self.vectorizer = None
        self.model = None
        self.category_mapping = {
            0: "activate_my_card",
            1: "age_limit",
            2: "apple_pay_or_google_pay",
            3: "atm_support",
            4: "automatic_top_up",
            5: "balance_not_updated_after_bank_transfer",
            6: "balance_not_updated_after_cheque_or_cash_deposit",
            7: "beneficiary_not_allowed",
            8: "cancel_transfer",
            9: "card_about_to_expire",
            10: "card_acceptance",
            11: "card_arrival",
            12: "card_delivery_estimate",
            13: "card_linking",
            14: "card_not_working",
            15: "card_payment_fee_charged",
            16: "card_payment_not_recognised",
            17: "card_payment_wrong_exchange_rate",
            18: "card_swallowed",
            19: "cash_withdrawal_charge",
            20: "cash_withdrawal_not_recognised",
            21: "change_pin",
            22: "compromised_card",
            23: "contactless_not_working",
            24: "country_support",
            25: "declined_card_payment",
            26: "declined_cash_withdrawal",
            27: "declined_transfer",
            28: "direct_debit_payment_not_recognised",
            29: "disposable_card_limits",
            30: "edit_personal_details",
            31: "exchange_charge",
            32: "exchange_rate",
            33: "exchange_via_app",
            34: "extra_charge_on_statement",
            35: "failed_transfer",
            36: "fiat_currency_support",
            37: "get_disposable_virtual_card",
            38: "get_physical_card",
            39: "getting_spare_card",
            40: "getting_virtual_card",
            41: "lost_or_stolen_card",
            42: "lost_or_stolen_phone",
            43: "order_physical_card",
            44: "passcode_forgotten",
            45: "pending_card_payment",
            46: "pending_cash_withdrawal",
            47: "pending_top_up",
            48: "pending_transfer",
            49: "pin_blocked",
            50: "receiving_money",
            51: "refund_not_showing_up",
            52: "request_refund",
            53: "reverted_card_payment",
            54: "supported_cards_and_currencies",
            55: "terminate_account",
            56: "top_up_by_bank_transfer_charge",
            57: "top_up_by_card_charge",
            58: "top_up_by_cash_or_cheque",
            59: "top_up_failed",
            60: "top_up_limits",
            61: "top_up_reverted",
            62: "topping_up_by_card",
            63: "transaction_charged_twice",
            64: "transfer_fee_charged",
            65: "transfer_into_account",
            66: "transfer_not_received_by_recipient",
            67: "transfer_timing",
            68: "unable_to_verify_identity",
            69: "verify_my_identity",
            70: "verify_source_of_funds",
            71: "verify_top_up",
            72: "virtual_card_not_working",
            73: "visa_or_mastercard",
            74: "why_verify_identity",
            75: "wrong_amount_of_cash_received",
            76: "wrong_exchange_rate_for_cash_withdrawal"
        }
        self.reverse_category_mapping = {v: k for k, v in self.category_mapping.items()}
        self.conversation_state = {
            'current_intent': None,
            'follow_up_count': 0,
            'context': {}
        }
        self.responses = {
            0: "To activate your new card, you can:\n1. Open the mobile app and go to the 'Cards' section\n2. Select 'Activate Card' and follow the on-screen instructions\n3. Or, you can call our customer support line and they'll be happy to assist you.",
            1: "To ensure you're eligible, please check our age requirements on the website or contact support for assistance.",
            2: "You can use Apple Pay or Google Pay by linking your card in the respective app settings.",
            3: "For ATM support, please visit our nearest branch or contact customer service.",
            4: "To set up automatic top-up, visit the app and choose 'Automatic Top-Up' under account settings.",
            5: "If your balance isn't updated after a bank transfer, please wait a few hours or contact support.",
            6: "Please check with your bank regarding cheque or cash deposit updates.",
            7: "If a beneficiary is not allowed, please review our beneficiary policy or contact customer support.",
            8: "To cancel a transfer, please go to the 'Transfers' section in the app and select 'Cancel'.",
            9: "If your card is about to expire, you will receive a new card automatically before the expiration date.",
            10: "To check card acceptance, please refer to our list of supported merchants on our website.",
            11: "Your card will arrive in 5-7 business days via standard mail.",
            12: "You can expect your card delivery to take up to 10 business days.",
            13: "Follow the app instructions to link your card, or contact support for help.",
            14: "If your card isn't working, please check for any alerts in the app or contact support.",
            15: "Any fees related to card payments will be detailed in your monthly statement.",
            16: "If your card payment isn't recognized, please verify your transaction history in the app.",
            17: "If you believe there’s a wrong exchange rate, please contact our customer support for clarification.",
            18: "If your card was swallowed by an ATM, please report it immediately to customer support.",
            19: "Cash withdrawal charges apply based on your account type; please refer to our fees page.",
            20: "If your cash withdrawal isn't recognized, please check your transaction history or contact support.",
            21: "To change your PIN, please visit the 'Security' section in the app.",
            22: "If you believe your card is compromised, please freeze it in the app and contact support.",
            23: "If your contactless payment isn’t working, please ensure your card is enabled for contactless.",
            24: "For country-specific support, please visit our support page for more details.",
            25: "If your card payment was declined, please check for any alerts or contact support.",
            26: "For declined cash withdrawals, please check your balance and limits in the app.",
            27: "If your transfer was declined, please check the reason in the app or contact support.",
            28: "To verify direct debit payments, please check your account settings or contact support.",
            29: "Disposable card limits can be set in the app under card settings.",
            30: "To edit personal details, go to your profile in the app and make the necessary changes.",
            31: "Exchange charges will be shown during the transaction process.",
            32: "Current exchange rates can be viewed in the app or on our website.",
            33: "You can exchange currency via the app in the 'Exchange' section.",
            34: "Extra charges will be detailed in your monthly statement.",
            35: "If a transfer has failed, please check the details and try again, or contact support.",
            36: "We support various fiat currencies; please check our website for details.",
            37: "To get a disposable virtual card, you can generate one through the app.",
            38: "To get a physical card, please request one in the app under 'Cards'.",
            39: "If you need a spare card, you can request one through the app.",
            40: "You can get a virtual card through the app options.",
            41: "If your card is lost or stolen, please report it immediately through the app.",
            42: "If your phone is lost or stolen, please contact support to secure your account.",
            43: "To order a physical card, please visit the 'Cards' section in the app.",
            44: "If you forgot your passcode, please follow the recovery steps in the app.",
            45: "If your card payment is pending, please check your transaction status in the app.",
            46: "Pending cash withdrawals may take a few moments to process.",
            47: "Pending top-ups can take time; please check your account for updates.",
            48: "Pending transfers will be shown in your transaction history.",
            49: "If your PIN is blocked, please follow the reset instructions in the app.",
            50: "For receiving money, please check the 'Receive' section in the app.",
            51: "If your refund isn't showing up, please verify with the merchant or contact support.",
            52: "To request a refund, please contact support with your transaction details.",
            53: "If your card payment was reverted, please check the transaction history.",
            54: "We support various cards and currencies; please refer to our website.",
            55: "To terminate your account, please follow the instructions in the app.",
            56: "A charge may apply for topping up by bank transfer depending on your account type. Please check our fees page for details.",
            57: "Top-ups made by card may have a fee depending on your card provider. Please check with them or refer to our fee policy.",
            58: "To top up by cash or cheque, please visit a participating branch or refer to our cash top-up options in the app.",
            59: "If your top-up has failed, please double-check the details and try again. Contact support if the issue persists.",
            60: "Top-up limits vary by account type. Please check your app settings under 'Top-Up Limits' or contact support.",
            61: "If your top-up was reverted, please confirm the reason in your transaction history or reach out to support.",
            62: "To top up using a card, please go to the 'Top-Up' section in the app and select 'Top-Up by Card'.",
            63: "If you were charged twice for a transaction, please verify your transaction history and contact support for assistance.",
            64: "Transfer fees are charged based on the currency and transfer type. Please refer to our fees page for more information.",
            65: "To transfer money into your account, please go to the 'Transfer' section in the app for account details.",
            66: "If your transfer hasn't been received by the recipient, please verify the details and contact support if needed.",
            67: "Transfer timing varies by currency and region. Check our website or app for estimated transfer times.",
            68: "If you're unable to verify your identity, please ensure your documents are correct and contact support if needed.",
            69: "To verify your identity, please follow the steps in the app under 'Verify Identity'.",
            70: "To verify the source of funds, please provide supporting documents as requested in the app.",
            71: "To verify your top-up, please follow any prompts in the app or contact support if verification is required.",
            72: "If your virtual card is not working, please check your app settings or contact support for help.",
            73: "We support Visa and Mastercard. Please check the app for card-specific options.",
            74: "We ask to verify your identity to comply with regulations and ensure account security.",
            75: "If you received the wrong amount of cash, please check your transaction history and contact support immediately.",
            76: "If you believe the exchange rate is incorrect for your cash withdrawal, please reach out to support for assistance."
        
        }
        self.model_dir = '/kaggle/working/saved_models'
        self.data_dir = '/kaggle/input/banking77'  
        os.makedirs(self.model_dir, exist_ok=True)
        self.initialize_nltk()
        
    def initialize_nltk(self):
        print("Initializing NLTK resources...")
        try:
            nltk.download('punkt', quiet=True)
            nltk.download('stopwords', quiet=True)
            self.stop_words = set(stopwords.words('english'))
            print("NLTK resources initialized successfully!")
        except Exception as e:
            print(f"Warning: Error initializing NLTK resources: {str(e)}")
            self.stop_words = set(['i', 'me', 'my', 'myself', 'we', 'our'])

    def load_dataset(self):
        print("Loading the Banking77 dataset from Hugging Face...")
        try:
            dataset = load_dataset("PolyAI/banking77")
            df = pd.DataFrame(dataset['train'])
            
            # Ensure the labels correspond to the correct indices
            unique_labels = sorted(df['label'].unique())
            self.category_mapping = {i: label for i, label in enumerate(unique_labels)}
            self.reverse_category_mapping = {label: i for i, label in self.category_mapping.items()}
            
            print(f"Dataset loaded successfully! Number of samples: {len(df)}")
            return df
            
        except Exception as e:
            print(f"Error loading dataset: {str(e)}")
            raise
            
    def save_model(self):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        try:
            model_path = os.path.join(self.model_dir, f'model_{timestamp}.joblib')
            vectorizer_path = os.path.join(self.model_dir, f'vectorizer_{timestamp}.joblib')
            mappings_path = os.path.join(self.model_dir, f'mappings_{timestamp}.json')
            
            joblib.dump(self.model, model_path)
            joblib.dump(self.vectorizer, vectorizer_path)
            
            mappings = {
                'category_mapping': {str(k): v for k, v in self.category_mapping.items()},
                'reverse_category_mapping': {k: int(v) for k, v in self.reverse_category_mapping.items()}
            }
            with open(mappings_path, 'w') as f:
                json.dump(mappings, f)
            
            latest_paths = {
                'model': model_path,
                'vectorizer': vectorizer_path,
                'mappings': mappings_path
            }
            with open(os.path.join(self.model_dir, 'latest_paths.json'), 'w') as f:
                json.dump(latest_paths, f)
            
            print(f"Model and components saved successfully in {self.model_dir}")
            return True
        except Exception as e:
            print(f"Error saving model: {str(e)}")
            return False
        
    def load_model(self):
        try:
            latest_paths_file = os.path.join(self.model_dir, 'latest_paths.json')
            if not os.path.exists(latest_paths_file):
                print("No saved model found. Please train the model first.")
                return False
            
            with open(latest_paths_file, 'r') as f:
                latest_paths = json.load(f)
            
            self.model = joblib.load(latest_paths['model'])
            self.vectorizer = joblib.load(latest_paths['vectorizer'])
            
            with open(latest_paths['mappings'], 'r') as f:
                mappings = json.load(f)
                self.category_mapping = {int(k): v for k, v in mappings['category_mapping'].items()}
                self.reverse_category_mapping = mappings['reverse_category_mapping']
            
            print("Model and components loaded successfully!")
            return True
        except Exception as e:
            print(f"Error loading model: {str(e)}")
            return False

    def preprocess_text(self, text):
        try:
            if pd.isna(text) or text is None:
                return ""
            text = str(text).lower()
            text = re.sub(r'[^a-z0-9\s?.!,]', '', text)
            tokens = word_tokenize(text)
            
            important_words = {'how', 'what', 'why', 'where', 'when', 'who', 'card', 'money', 'transfer', 'receive', 'exchange', 'rate'}
            tokens = [token for token in tokens if token not in self.stop_words or token in important_words]
            
            processed_text = " ".join(tokens)
            return processed_text
        except Exception as e:
            print(f"Error during text preprocessing: {str(e)}")
            return text

    def train_model(self, df):
        try:
            X = df['text'].apply(self.preprocess_text)
            y = df['label'].map(self.reverse_category_mapping)

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            self.vectorizer = TfidfVectorizer()
            X_train_vectorized = self.vectorizer.fit_transform(X_train)
            self.model = LogisticRegression(max_iter=200)
            self.model.fit(X_train_vectorized, y_train)
            accuracy = self.model.score(self.vectorizer.transform(X_test), y_test)
            
            print(f"Model trained successfully! Accuracy: {accuracy:.2f}")
            return accuracy
        except Exception as e:
            print(f"Error during model training: {str(e)}")
            return None

    def predict(self, input_text):
        try:
            if not self.model or not self.vectorizer:
                print("Model or vectorizer not loaded. Please load the model first.")
                return None

            # Preprocess the input text
            processed_input = self.preprocess_text(input_text)

            # Transform the processed input to vector format
            X_input = self.vectorizer.transform([processed_input])

            # Make a prediction
            predicted_label = self.model.predict(X_input)

            # Get the predicted category index
            predicted_category_index = predicted_label[0]  # Get the predicted label

            # Get the predicted category name from the mapping
            predicted_category_name = self.category_mapping.get(predicted_category_index, "Unknown Category")

            # Get the corresponding response
            response = self.responses.get(predicted_category_index, "I'm sorry, I can't assist with that request.")

            return predicted_category_name, response  # Return the category name and response
        except Exception as e:
            print(f"Error during prediction: {str(e)}")
            return None, None

def test_chatbot_notebook():
    chatbot = CustomerSupportChatbot()
    
    # Load dataset and train the model
    df = chatbot.load_dataset()
    chatbot.train_model(df)

    # Save the model
    chatbot.save_model()
    
    # Load the model
    chatbot.load_model()

    # Test predictions
    test_inputs = [
        "How can I activate my card?",
        "I want to know about the cash withdrawal charges.",
        "What do I do if my card is lost?",
        "I was charged twice for a transaction.",
        "How do I cancel a transfer?"
    ]

    for input_text in test_inputs:
        print(f"You: {input_text}")
        predicted_category_name, response = chatbot.predict(input_text)
        print(f"Predicted category: {predicted_category_name}\nResponse: {response}\n")

# Uncomment to run the test
test_chatbot_notebook()

Initializing NLTK resources...
NLTK resources initialized successfully!
Loading the Banking77 dataset from Hugging Face...


banking77.py:   0%|          | 0.00/7.17k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/5.89k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.78k [00:00<?, ?B/s]

The repository for PolyAI/banking77 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/banking77.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/10003 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3080 [00:00<?, ? examples/s]

Dataset loaded successfully! Number of samples: 10003
Model trained successfully! Accuracy: 0.86
Error saving model: Object of type int64 is not JSON serializable
No saved model found. Please train the model first.
You: How can I activate my card?
Predicted category: 0
Response: To activate your new card, you can:
1. Open the mobile app and go to the 'Cards' section
2. Select 'Activate Card' and follow the on-screen instructions
3. Or, you can call our customer support line and they'll be happy to assist you.

You: I want to know about the cash withdrawal charges.
Predicted category: 20
Response: If your cash withdrawal isn't recognized, please check your transaction history or contact support.

You: What do I do if my card is lost?
Predicted category: 41
Response: If your card is lost or stolen, please report it immediately through the app.

You: I was charged twice for a transaction.
Predicted category: 63
Response: I'm sorry, I can't assist with that request.

You: How do I cancel a